# FCLayer의 Backpropagation을 이용한 weight update실습

다음과 같은 내용을 손글씨 내용정리를 통하여 알고 있습니다!

## Feedforward

$Input : a^\ell$ \
$z^\ell = w^\ell a^{\ell-1} + b^\ell$ and $a^\ell = \sigma(z^\ell)$ &nbsp; where &nbsp; $\ell = 2,3, ... L$ 


## Backward
output error : $\delta^l = {\partial L\over\partial z^\ell} = {\partial L\over\partial a^\ell} {\partial a^\ell\over\partial z^\ell}$ &nbsp; where &nbsp; $L$ : Loss \
$\delta^\ell = (w^\ell)^Td^{\ell+1} \circledcirc \sigma(z^\ell)$ &nbsp; where &nbsp; $\ell = (\ell-1), (\ell-2), ... 2$
* 이때 실습에서는 MSELoss를 사용했습니다. \
MSELoss = $\frac{1}{N} \sum_{i=1}^{N}(a^L_i-t_i)^2$ &nbsp; where &nbsp; $t = target$ 


## weight update
$w^\ell$ -> $w^\ell - lr {\partial L\over\partial w^\ell}$ &nbsp; where &nbsp; ${\partial L\over\partial w^\ell} = \delta^\ell (a^{\ell-1})^T$ \
$b^\ell$ -> $b^\ell - lr {\partial L\over\partial b^\ell}$ &nbsp; where &nbsp; ${\partial L\over\partial b^\ell} = \delta^\ell$

# (문제) numpy를 이용한 NumpyNet을 구현 하세요
- 위 내용을 참고하여 아래의 빈칸을 채워 넣으세요
- 아래 파이토치 모델과 동일하게 weight update하는 모델을 만드는 것이 목표입니다.
- 아래의 예제문제를 통하여 weight를 확인해 보세요!

예제 1 : 2layer model입니다. weight, bias로 이루어져 있습니다. \
예제 2 : 2layer model입니다. weight, bias, activation로 이루어져 있습니다.  \
예제 3 : 3layer model입니다. Batch학습이 가능하도록 설계해봅니다. \
예제 4 : 3layer model입니다. Batch학습과 output이 2차원 일때 테스트 해봅니다!
- 자세한 내용은 각 문제를 확인해주세요!
- 아래의 정답 유무에서는 pytorch 모델과 근사치를 구하는 것이 다를 수 있으므로 오차를 $10^{-6}$ 범위내에서는 같다고 판단하였습니다.

In [1]:
import numpy as np

class NumpyNet:
    def __init__(self, Weight, Bias, Activation=lambda x: x):
        assert len(Weight) == len(Bias), "prdict와 target의 길이가 같아야합니다."
        self.weight = [np.array(w, dtype=np.float64) for w in Weight]
        self.bias = [np.array(b, dtype=np.float64) for b in Bias]
        self.f = Activation

    def __call__(self, x):
        return self.forward(x)

    def forward(self, x):
        if not isinstance(type(x), np.ndarray):
            x = np.array(x, dtype=np.float64)

        self.forward_result = [x]
        for w, b in zip(self.weight, self.bias):
            z = w @ x + b
            x = self.f(z)
            self.forward_result.append(z)
        return x.squeeze()

    def backward(self, predict, target, h=1e-8):
        assert len(predict) == len(target), "prdict와 target의 길이가 같아야합니다."

        loss = self.mse_loss(predict, target)
        for idx in range(len(self.weight) - 1, 0, -1):
            w = self.weight[idx]
            d = self.backward_result[0]
            z = self.forward_result[idx]
            new_d = (w.T @ d) * self.diff(self.f, z, h)
            self.backward_result.insert(0, new_d)
        return loss

    def weight_update(self, lr=0.01):
        if len(self.forward_result[0].shape) == 1:
            fr = [np.expand_dims(f, 0) for f in self.forward_result[:-1]]
            br = [np.expand_dims(b, 1) for b in self.backward_result]
            self.weight = [w - lr * (b @ self.f(z)) for w, z, b in zip(self.weight, fr, br)]
            self.bias = [bia - lr * b.squeeze() for bia, b in zip(self.bias, br)]
        else:
            fr = self.forward_result[:-1]
            br = self.backward_result
            self.weight = [w - lr * (b @ self.f(z.T)) for w, z, b in zip(self.weight, fr, br)]
            self.bias = [bia - lr * b.sum(axis=1, keepdims=True) for bia, b in zip(self.bias, br)]

    def mse_loss(self, predict, target, h=1e-8):
        if not isinstance(np.ndarray, type(target)):
            target = np.array(target, dtype=np.float64)

        eps = np.identity(len(target)) * h
        if len(target.shape) == 1:
            loss_rh = self.mse((predict + eps), target)
            loss_lh = self.mse((predict - eps), target)
            gap = np.diag(loss_rh - loss_lh)
        else:
            predict = np.expand_dims(predict.T, 1)
            target = np.expand_dims(target.T, 1)
            loss_rh = self.mse((predict + eps), target).squeeze()
            loss_lh = self.mse((predict - eps), target).squeeze()
            gap = np.diagonal(loss_rh - loss_lh, axis1=1, axis2=2).T

        d = gap / (2 * h) * self.diff(self.f, self.forward_result[-1], h)
        self.backward_result = [d]

        loss = self.mse(predict, target)
        return loss.sum()

    def mse(self, predict, target):
        result = np.divide(np.power(predict - target, 2), np.product(target.shape))
        return result

    def diff(self, f, x, h):
        result = (f(x + h) - f(x - h)) / (2 * h)
        return result

## pytorch를 이용한 TorchNet

In [2]:
import torch
from torch import nn
from torch import optim

class TorchNet(nn.Module):
    def __init__(self, Weight, Bias, Activation=False):
        super().__init__()
        self.net = nn.ModuleList()
        for w, b in zip(Weight, Bias):
            w = torch.tensor(w, dtype=torch.float64)
            b = torch.tensor(b, dtype=torch.float64)
            fc = nn.Linear(*torch.t(w).size())
            fc.weight = nn.Parameter(w)
            fc.bias = nn.Parameter(b.squeeze())
            self.net.append(fc)
            if Activation:
                self.net.append(Activation())

    def forward(self, x):
        if not isinstance(torch.tensor, type(x)):
            x = torch.tensor(x, dtype=torch.float64)
        for layer in self.net:
            x = layer(x)
        return x

# 예제1
- input, target, weight, bias를 표현하면 다음과 같습니다.

$a^1 = \begin{pmatrix} 1 \\ 1 \end{pmatrix}$ &nbsp; and &nbsp;  $t = \begin{pmatrix} 2 \\ 3 \end{pmatrix}$ \
$w^2 = \begin{pmatrix} 1 & -2 \\ 2 & 4 \\ -3 & 1 \end{pmatrix}$ &nbsp; and &nbsp; $b^2 = \begin{pmatrix} 1 \\ 2 \\ 3 \end{pmatrix}$ \
$w^3 = \begin{pmatrix} 1 & 2 & -3 \\ 2 & -1 & 3 \end{pmatrix}$ &nbsp; and &nbsp; $b^3 = \begin{pmatrix} 2 \\ 1 \end{pmatrix}$ \
$activation function : \sigma(x) = x$


In [3]:
input_data, target = [1, 1], [2, 3]

weight1 = [[1, -2], [2, 4], [-3, 1]]
weight2 = [[1, 2, -3], [2, -1, 3]]
weight = [weight1, weight2]

bias1 = [1, 2, 3]
bias2 = [2, 1]
bias = [bias1, bias2]

## 학습 결과 비교

In [4]:
print("넘파이 학습")
numpy_net = NumpyNet(Weight=weight, Bias=bias)
for epoch in range(1):
    outputs = numpy_net(input_data)
    numpy_net.backward(outputs, target)
    numpy_net.weight_update()

print([f"{i:.4f}" for i in sum(map(list,numpy_net.weight[0]),[])])
print([f"{i:.4f}" for i in sum(map(list,numpy_net.weight[1]),[])])
print([f"{i:.4f}" for i in numpy_net.bias[0]])
print([f"{i:.4f}" for i in numpy_net.bias[1]])

print("파이토치 학습")
torch_net = TorchNet(Weight=weight, Bias=bias)
criterion = nn.MSELoss()
optimizer = optim.SGD(torch_net.parameters(), lr=0.01)
for epoch in range(1):
    optimizer.zero_grad()
    outputs = torch_net(input_data)
    loss = criterion(outputs, torch.tensor(target, dtype=torch.float64))
    loss.backward()
    optimizer.step()

print([f"{i:.4f}" for i in sum(map(list,torch_net.net[0].weight),[])])
print([f"{i:.4f}" for i in sum(map(list,torch_net.net[1].weight),[])])
print([f"{i:.4f}" for i in torch_net.net[0].bias])
print([f"{i:.4f}" for i in torch_net.net[1].bias])

print("\n")
result1 = (numpy_net.weight[0] - torch_net.net[0].weight.detach().numpy() <= 1e-6).all()
result2 = (numpy_net.weight[1] - torch_net.net[1].weight.detach().numpy() <= 1e-6).all()
result3 = (numpy_net.bias[0] - torch_net.net[0].bias.detach().numpy() <= 1e-6).all()
result4 = (numpy_net.bias[1] - torch_net.net[1].bias.detach().numpy() <= 1e-6).all()
print("정답 입니다!" if all([result1,result2,result3,result4]) else "오답 입니다!")

넘파이 학습
['1.0100', '-1.9900', '1.6700', '3.6700', '-2.4000', '1.6000']
['1.0000', '0.9600', '-3.1300', '2.0000', '-0.4400', '3.0700']
['1.0100', '1.6700', '3.6000']
['1.8700', '1.0700']
파이토치 학습
['1.0100', '-1.9900', '1.6700', '3.6700', '-2.4000', '1.6000']
['1.0000', '0.9600', '-3.1300', '2.0000', '-0.4400', '3.0700']
['1.0100', '1.6700', '3.6000']
['1.8700', '1.0700']


정답 입니다!


# 예제 2
- input, target, weight, bias를 표현하면 다음과 같습니다.
- 예제 1과 다르게 activation이 추가되었습니다.

$a^1 = \begin{pmatrix} 1 \\ 1 \end{pmatrix}$ &nbsp; and &nbsp;  $t = \begin{pmatrix} 2 \\ 3 \end{pmatrix}$ \
$w^2 = \begin{pmatrix} 1 & -2 \\ 1 & 1 \end{pmatrix}$ &nbsp; and &nbsp; $b^2 = \begin{pmatrix} 0 \\ 0 \end{pmatrix}$ \
$w^3 = \begin{pmatrix} 1 & -1 \\ 2 & -1 \end{pmatrix}$ &nbsp; and &nbsp; $b^3 = \begin{pmatrix} 0 \\ 0 \end{pmatrix}$ \
$activation function : \sigma(x) = x^2$



In [5]:
input_data, target = [1, 1], [2, 3]

weight1 = [[1, -2], [1, 1]]
weight2 = [[1, -1], [2, -1]]
weight = [weight1, weight2]

bias1 = [0, 0]
bias2 = [0, 0]
bias = [bias1, bias2]

## 학습 결과

In [6]:
print("넘파이 학습")
numpy_net = NumpyNet(Weight=weight, Bias=bias, Activation=lambda x: x**2)
for epoch in range(1):
    outputs = numpy_net(input_data)
    numpy_net.backward(outputs, target)
    numpy_net.weight_update()

print([f"{i:.4f}" for i in sum(map(list,numpy_net.weight[0]),[])])
print([f"{i:.4f}" for i in sum(map(list,numpy_net.weight[1]),[])])
print([f"{i:.4f}" for i in numpy_net.bias[0]])
print([f"{i:.4f}" for i in numpy_net.bias[1]])

class SqureActivation(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x**2

print("파이토치 학습")
active = SqureActivation
torch_net = TorchNet(weight, bias, active)
criterion = nn.MSELoss()
optimizer = optim.SGD(torch_net.parameters(), lr=0.01)
for epoch in range(1):
    optimizer.zero_grad()
    outputs = torch_net(input_data)
    loss = criterion(outputs, torch.tensor(target, dtype=torch.float64))
    loss.backward()
    optimizer.step()

print([f"{i:.4f}" for i in sum(map(list,torch_net.net[0].weight),[])])
print([f"{i:.4f}" for i in sum(map(list,torch_net.net[2].weight),[])])
print([f"{i:.4f}" for i in torch_net.net[0].bias])
print([f"{i:.4f}" for i in torch_net.net[2].bias])

print("\n")
result1 = (numpy_net.weight[0] - torch_net.net[0].weight.detach().numpy() <= 1e-6).all()
result2 = (numpy_net.weight[1] - torch_net.net[2].weight.detach().numpy() <= 1e-6).all()
result3 = (numpy_net.bias[0] - torch_net.net[0].bias.detach().numpy() <= 1e-6).all()
result4 = (numpy_net.bias[1] - torch_net.net[2].bias.detach().numpy() <= 1e-6).all()
print("정답 입니다!" if all([result1,result2,result3,result4]) else "오답 입니다!")

넘파이 학습
['-0.0000', '-3.0000', '-0.8400', '-0.8400']
['1.4200', '0.6800', '2.0400', '-0.8400']
['-1.0000', '-1.8400']
['0.4200', '0.0400']
파이토치 학습
['0.0000', '-3.0000', '-0.8400', '-0.8400']
['1.4200', '0.6800', '2.0400', '-0.8400']
['-1.0000', '-1.8400']
['0.4200', '0.0400']


정답 입니다!


# 예제 3
- Batch 학습이 가능하도록 설계해보세요!
- input data와 target은 다음과 같이 한 쌍입니다. $(a^1_i,t_i)$ where $i = 1, 2, 3$
- input data 행렬에 대하여 pytorch는 행을 기준으로 서로 다른 데이터임을 구분하지만 주어진 input data는 열을 기준으로 하고 있습니다.

$a^1_1 = \begin{pmatrix} 1 \\ 2 \end{pmatrix}$, &nbsp; $a^1_2 = \begin{pmatrix} 1 \\ -1 \end{pmatrix}$, &nbsp; $a^1_3 = \begin{pmatrix} 1 \\ 1 \end{pmatrix}$, &nbsp; -> &nbsp; $a^1 = \begin{pmatrix} 1 & 1 & 1 \\ 2 & -1 & 1 \end{pmatrix}$ 

$t_1 = \begin{pmatrix} -1 \end{pmatrix}$, &nbsp; $t_2 = \begin{pmatrix} 0 \end{pmatrix}$, &nbsp; $t_3 = \begin{pmatrix} 1 \end{pmatrix}$, &nbsp; -> &nbsp; $t = \begin{pmatrix} -1 & 0 & 1 \end{pmatrix}$ 

$w^2 = \begin{pmatrix} 1 & 2 \\ 1 & -1 \\ 2 & 3 \end{pmatrix}$ &nbsp; and &nbsp; $b^2 = \begin{pmatrix} 0 \\ 0 \\ 0 \end{pmatrix}$ 

$w^3 = \begin{pmatrix} 2 & 3 & 1 \\ 2 & -1 & 2 \end{pmatrix}$ &nbsp; and &nbsp; $b^3 = \begin{pmatrix} 0 \\ 0 \end{pmatrix}$ 

$w^4 = \begin{pmatrix} 1 & 1 \end{pmatrix}$ &nbsp; and &nbsp; $b^4 = \begin{pmatrix} 0 \end{pmatrix}$ 

$activation function : \sigma(x) = x$


In [7]:
input_data = [[1, 1, 1], [2, -1, 1]]
target = [-1, 0, 1]

weight1 = [[1, 2], [2, 1], [-1, 2]]
weight2 = [[2, 3, 1], [2, -1, 2]]
weight3 = [[1, 1]]
weight = [weight1, weight2, weight3]

bias1 = [[0], [0], [0]]
bias2 = [[0], [0]]
bias3 = [[0]]
bias = [bias1, bias2, bias3]

## 학습결과

In [8]:
print("넘파이 학습")
numpy_net = NumpyNet(weight, bias)
for epoch in range(1):
    outputs = numpy_net(input_data)
    numpy_net.backward(outputs, target)
    numpy_net.weight_update()

print([f"{i:.4f}" for i in sum(map(list, numpy_net.weight[0]), [])])
print([f"{i:.4f}" for i in sum(map(list, numpy_net.weight[1]), [])])
print([f"{i:.4f}" for i in sum(map(list, numpy_net.weight[2]), [])])
print([f"{i:.4f}" for i in sum(map(list, numpy_net.bias[0]), [])])
print([f"{i:.4f}" for i in sum(map(list, numpy_net.bias[1]), [])])
print([f"{i:.4f}" for i in sum(map(list, numpy_net.bias[2]), [])])

print("파이토치 학습")
torch_net = TorchNet(weight, bias)
criterion = nn.MSELoss()
optimizer = optim.SGD(torch_net.parameters(), lr=0.01)
for epoch in range(1):
    optimizer.zero_grad()
    outputs = torch_net(np.array(input_data).T)
    loss = criterion(outputs, torch.tensor([target], dtype=torch.float64).T)
    loss.backward()
    optimizer.step()

print([f"{i:.4f}" for i in sum(map(list, torch_net.net[0].weight), [])])
print([f"{i:.4f}" for i in sum(map(list, torch_net.net[1].weight), [])])
print([f"{i:.4f}" for i in sum(map(list, torch_net.net[2].weight), [])])
print([f"{i:.4f}" for i in torch_net.net[0].bias])
print([f"{i:.4f}" for i in torch_net.net[1].bias])
print([f"{float(torch_net.net[2].bias):.4f}"])

print("\n")
result1 = (numpy_net.weight[0] - torch_net.net[0].weight.detach().numpy() <= 1e-6).all()
result2 = (numpy_net.weight[1] - torch_net.net[1].weight.detach().numpy() <= 1e-6).all()
result3 = (numpy_net.weight[2] - torch_net.net[2].weight.detach().numpy() <= 1e-6).all()
result4 = (numpy_net.bias[0].squeeze() - torch_net.net[0].bias.detach().numpy() <= 1e-6).all()
result5 = (numpy_net.bias[1].squeeze() - torch_net.net[1].bias.detach().numpy() <= 1e-6).all()
result6 = (numpy_net.bias[2].squeeze() - torch_net.net[2].bias.detach().numpy() <= 1e-6).all()
print("정답 입니다!" if all([result1,result2,result3,result4,result5,result6]) else "오답 입니다!")

넘파이 학습
['-0.2533', '-0.8533', '1.3733', '-0.4267', '-1.9400', '-0.1400']
['0.2600', '1.6600', '-0.1133', '0.2600', '-2.3400', '0.8867']
['-7.6133', '-3.3667']
['-1.2533', '-0.6267', '-0.9400']
['-0.3133', '-0.3133']
['-0.3133']
파이토치 학습
['-0.2533', '-0.8533', '1.3733', '-0.4267', '-1.9400', '-0.1400']
['0.2600', '1.6600', '-0.1133', '0.2600', '-2.3400', '0.8867']
['-7.6133', '-3.3667']
['-1.2533', '-0.6267', '-0.9400']
['-0.3133', '-0.3133']
['-0.3133']


정답 입니다!


# 예제 4

- input data와 target은 다음과 같이 한 쌍입니다. $(a^1_i,t_i)$ where $i = 1, 2, 3$

$a^1_1 = \begin{pmatrix} 1 \\ 2 \end{pmatrix}$, &nbsp; $a^1_2 = \begin{pmatrix} 1 \\ -1 \end{pmatrix}$, &nbsp; $a^1_3 = \begin{pmatrix} 1 \\ 1 \end{pmatrix}$, &nbsp; -> &nbsp; $a^1 = \begin{pmatrix} 1 & 1 & 1 \\ 2 & -1 & 1 \end{pmatrix}$ 

$t_1 = \begin{pmatrix} -1 \\ 2 \end{pmatrix}$, &nbsp; $t_2 = \begin{pmatrix} 0 \\ 1 \end{pmatrix}$, &nbsp; $t_3 = \begin{pmatrix} 1 \\ 1 \end{pmatrix}$, &nbsp; -> &nbsp; $t = \begin{pmatrix} -1 & 0 & 1 \\ 2 & 1 & 1 \end{pmatrix}$ 

$w^2 = \begin{pmatrix} 1 & 2 \\ 2 & 1 \\ -1 & 2 \end{pmatrix}$ &nbsp; and &nbsp; $b^2 = \begin{pmatrix} 0 \\ 0 \\ 0 \end{pmatrix}$ 

$w^3 = \begin{pmatrix} 2 & 3 & 1 \\ 2 & -1 & 2 \end{pmatrix}$ &nbsp; and &nbsp; $b^3 = \begin{pmatrix} 0 \\ 0 \end{pmatrix}$ 

$w^4 = \begin{pmatrix} 1 & -1 \\ 2 & 1 \end{pmatrix}$ &nbsp; and &nbsp; $b^4 = \begin{pmatrix} 0 \\ 0 \end{pmatrix}$ 

$activation function : \sigma(x) = x$

- output이 2차원 데이터 이므로 Loss를 다음과 같이 계산해야 합니다.
- MSELoss = $\frac{1}{N*M} \sum_{i=1}^{N}\sum_{j=1}^{M}(a^L_{i,j}-t_{i,j})^2$ &nbsp; where &nbsp; $t = target$ 


In [9]:
input_data = [[1, 1, 1], [2, -1, 1]]
target = [[-1, 0, 1], [2, 1, 1]]

weight1 = [[1, 2], [2, 1], [-1, 2]]
weight2 = [[2, 3, 1], [2, -1, 2]]
weight3 = [[1, -1], [2, 1]]
weight = [weight1, weight2, weight3]

bias1 = [[0], [0], [0]]
bias2 = [[0], [0]]
bias3 = [[0], [0]]
bias = [bias1, bias2, bias3]


## 학습결과

In [10]:
print("넘파이 학습")
numpy_net = NumpyNet(weight, bias)
for epoch in range(1):
    outputs = numpy_net(input_data)
    numpy_net.backward(outputs, target)
    numpy_net.weight_update()

print([f"{i:.4f}" for i in sum(map(list, numpy_net.weight[0]), [])])
print([f"{i:.4f}" for i in sum(map(list, numpy_net.weight[1]), [])])
print([f"{i:.4f}" for i in sum(map(list, numpy_net.weight[2]), [])])
print([f"{i:.4f}" for i in sum(map(list, numpy_net.bias[0]), [])])
print([f"{i:.4f}" for i in sum(map(list, numpy_net.bias[1]), [])])
print([f"{i:.4f}" for i in sum(map(list, numpy_net.bias[2]), [])])

print("파이토치 학습")
torch_net = TorchNet(weight, bias)
criterion = nn.MSELoss()
optimizer = optim.SGD(torch_net.parameters(), lr=0.01)
for epoch in range(1):
    optimizer.zero_grad()
    outputs = torch_net(np.array(input_data).T)
    loss = criterion(outputs, torch.tensor(target, dtype=torch.float64).T)
    loss.backward()
    optimizer.step()

print([f"{i:.4f}" for i in sum(map(list, torch_net.net[0].weight), [])])
print([f"{i:.4f}" for i in sum(map(list, torch_net.net[1].weight), [])])
print([f"{i:.4f}" for i in sum(map(list, torch_net.net[2].weight), [])])
print([f"{i:.4f}" for i in torch_net.net[0].bias])
print([f"{i:.4f}" for i in torch_net.net[1].bias])
print([f"{i:.4f}" for i in torch_net.net[2].bias])

print("\n")
result1 = (numpy_net.weight[0] - torch_net.net[0].weight.detach().numpy() <= 1e-6).all()
result2 = (numpy_net.weight[1] - torch_net.net[1].weight.detach().numpy() <= 1e-6).all()
result3 = (numpy_net.weight[2] - torch_net.net[2].weight.detach().numpy() <= 1e-6).all()
result4 = (numpy_net.bias[0].squeeze() - torch_net.net[0].bias.detach().numpy() <= 1e-6).all()
result5 = (numpy_net.bias[1].squeeze() - torch_net.net[1].bias.detach().numpy() <= 1e-6).all()
result6 = (numpy_net.bias[2].squeeze() - torch_net.net[2].bias.detach().numpy() <= 1e-6).all()
print("정답 입니다!" if all([result1,result2,result3,result4,result5,result6]) else "오답 입니다!")

넘파이 학습
['-0.6400', '-1.4000', '0.2200', '-2.2467', '-1.9900', '-0.1633']
['-1.1233', '0.4633', '-0.8233', '0.9033', '-1.8033', '1.2433']
['-0.6533', '-1.5167', '-5.0133', '-2.4200']
['-1.6400', '-1.7800', '-0.9900']
['-0.6500', '-0.1700']
['-0.1033', '-0.2733']
파이토치 학습
['-0.6400', '-1.4000', '0.2200', '-2.2467', '-1.9900', '-0.1633']
['-1.1233', '0.4633', '-0.8233', '0.9033', '-1.8033', '1.2433']
['-0.6533', '-1.5167', '-5.0133', '-2.4200']
['-1.6400', '-1.7800', '-0.9900']
['-0.6500', '-0.1700']
['-0.1033', '-0.2733']


정답 입니다!


# Version

- Python==3.8.18
- torch==1.13.1
- numpy==1.24.3